## ETL process

Distributions

Missing Data

Corelation Analysis

Data Transformation

## Classification

1. Logistic Regresion:

Justification: Powerful algorithm for binary classification problems like predicting survival in this case survived or not survided. Plus it is easily interpretable.

2. Random Forest:

Justification: Given the Titanic dataset’s diversity in features (like age, fare, class, etc.), Random Forest can effectively capture the non-linear relationships between features and survival.

In [ ]:
#Dany

3. Support Vector Machine (SVM):

Justification: SVM can efficiently classify data points even when the relationship between the features and the target variable is complex.

## K-cross validation